In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

import matplotlib.pyplot as plt
import numpy as np
import os
from os.path import join
import string
from cv2 import cv2
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from numpy import expand_dims
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from matplotlib import pyplot


from tensorflow.keras import layers

Import Tensorflow Datasets

In [3]:
(train_dsf, val_dsf, test_dsf), metadata = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

num_classes_dsf = metadata.features['label'].num_classes
print(num_classes_dsf)

5


In [ ]:
(train_dscd, val_dscd, test_dscd), metadata = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
    batch_size=64,
)

num_classes_dscd = metadata.features['label'].num_classes
print(num_classes_dscd)

2


Set up Obfuscation of data

In [4]:

IMG_SIZE = 180

resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(IMG_SIZE, IMG_SIZE),
  layers.experimental.preprocessing.Rescaling(1./255)
])

data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.2),
])

batch_size = 32
AUTOTUNE = tf.data.AUTOTUNE

def prepare(ds, shuffle=False, augment=False):
  # Resize and rescale all datasets
  ds = ds.map(lambda x, y: (resize_and_rescale(x), y), 
              num_parallel_calls=AUTOTUNE)

  if shuffle:
    ds = ds.shuffle(1000)

  # Batch all datasets
  ds = ds.batch(batch_size)

  # Use data augmentation only on the training set
  if augment:
    ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y), 
                num_parallel_calls=AUTOTUNE)

  # Use buffered prefecting on all datasets
  return ds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
train_ds = prepare(train_dsf, shuffle=True, augment=False)
val_ds = prepare(val_dsf)
test_ds = prepare(test_dsf, augment=False)
test_ds_aug = prepare(test_dsf, augment=True)

In [ ]:
#Model with no augmentation tested on augmented data set

model = tf.keras.Sequential([
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes_dsf)
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


epochs=5
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)



Epoch 1/5
92/92 [==============================] - 86s 909ms/step - loss: 1.2703 - accuracy: 0.4636 - val_loss: 1.1411 - val_accuracy: 0.5395
Epoch 2/5
92/92 [==============================] - 81s 879ms/step - loss: 0.9435 - accuracy: 0.6379 - val_loss: 0.9065 - val_accuracy: 0.6594
Epoch 3/5
92/92 [==============================] - 82s 885ms/step - loss: 0.7244 - accuracy: 0.7279 - val_loss: 0.8827 - val_accuracy: 0.7003
Epoch 4/5
92/92 [==============================] - 82s 891ms/step - loss: 0.5248 - accuracy: 0.8089 - val_loss: 1.0099 - val_accuracy: 0.6376
Epoch 5/5
92/92 [==============================] - 82s 889ms/step - loss: 0.3292 - accuracy: 0.8873 - val_loss: 0.9774 - val_accuracy: 0.7003


In [ ]:
loss, acc = model.evaluate(test_ds)
loss, acc = model.evaluate(test_ds_aug)

12/12 [==============================] - 4s 300ms/step - loss: 1.2291 - accuracy: 0.5940


Testing a Model with Obfuscated data

In [ ]:
train_ds = prepare(train_dsf, shuffle=True, augment=True)
val_ds = prepare(val_dsf, augment=True)
test_ds = prepare(test_dsf, augment=False)
test_ds_aug = prepare(test_dsf, augment=True)

In [ ]:
#Model with augmentation tested on augmented data set

model = tf.keras.Sequential([
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes_dsf)
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


epochs=5
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)


Epoch 1/5


ValueError: ignored

In [ ]:
loss, acc = model.evaluate(test_ds)
loss, acc = model.evaluate(test_ds_aug)

12/12 [==============================] - 4s 301ms/step - loss: 0.7927 - accuracy: 0.6594


Testing other Randmization in the data

In [ ]:
#Random Contrast

IMG_SIZE = 180
seed = 3

resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(IMG_SIZE, IMG_SIZE),
  layers.experimental.preprocessing.Rescaling(1./255)
])

#Augmentation Only RandomContrast
data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomContrast((.8,1.5),seed=seed),
])



In [ ]:
train_ds = prepare(train_dsf, shuffle=True, augment=False)
val_ds = prepare(val_dsf, augment=False)
test_ds = prepare(test_dsf, augment=False)
test_ds_aug = prepare(test_dsf, augment=True)

In [ ]:
#Model with augmentation tested on augmented data set

model = tf.keras.Sequential([
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes_dsf)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


epochs=5
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/5
92/92 [==============================] - 94s 988ms/step - loss: 1.2935 - accuracy: 0.4574 - val_loss: 1.0667 - val_accuracy: 0.5995
Epoch 2/5
92/92 [==============================] - 89s 957ms/step - loss: 0.9572 - accuracy: 0.6206 - val_loss: 0.9226 - val_accuracy: 0.6403
Epoch 3/5
92/92 [==============================] - 89s 958ms/step - loss: 0.7665 - accuracy: 0.7047 - val_loss: 1.0260 - val_accuracy: 0.5995
Epoch 4/5
92/92 [==============================] - 89s 959ms/step - loss: 0.5818 - accuracy: 0.7854 - val_loss: 1.0129 - val_accuracy: 0.6294
Epoch 5/5
92/92 [==============================] - 89s 963ms/step - loss: 0.4020 - accuracy: 0.8597 - val_loss: 1.0754 - val_accuracy: 0.7003


In [ ]:
loss, acc = model.evaluate(test_ds)
loss, acc = model.evaluate(test_ds_aug)

12/12 [==============================] - 3s 264ms/step - loss: 1.7732 - accuracy: 0.5477


In [ ]:
train_ds = prepare(train_dsf, shuffle=True, augment=True)
val_ds = prepare(val_dsf, augment=True)
test_ds = prepare(test_dsf, augment=False)
test_ds_aug = prepare(test_dsf, augment=True)

In [ ]:
#Model with augmentation tested on augmented data set

model = tf.keras.Sequential([
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes_dsf)
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


epochs=5
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)


Epoch 1/5
92/92 [==============================] - 90s 963ms/step - loss: 1.6020 - accuracy: 0.3123 - val_loss: 1.3813 - val_accuracy: 0.3597
Epoch 2/5
92/92 [==============================] - 89s 963ms/step - loss: 1.2194 - accuracy: 0.4843 - val_loss: 1.1706 - val_accuracy: 0.5395
Epoch 3/5
92/92 [==============================] - 89s 964ms/step - loss: 1.1282 - accuracy: 0.5453 - val_loss: 1.0997 - val_accuracy: 0.5722
Epoch 4/5
92/92 [==============================] - 89s 966ms/step - loss: 1.0228 - accuracy: 0.5923 - val_loss: 1.1572 - val_accuracy: 0.5477
Epoch 5/5
92/92 [==============================] - 89s 965ms/step - loss: 0.9464 - accuracy: 0.6257 - val_loss: 1.0311 - val_accuracy: 0.6104


In [ ]:
loss, acc = model.evaluate(test_ds)
loss, acc = model.evaluate(test_ds_aug)

12/12 [==============================] - 3s 267ms/step - loss: 0.9576 - accuracy: 0.6267


In [ ]:

#Augmentation Only RandomZoom
data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomZoom((-0.3, -0.2),seed=seed,fill_mode='wrap'),
])



In [ ]:
train_ds = prepare(train_dsf, shuffle=True, augment=False)
val_ds = prepare(val_dsf, augment=False)
test_ds = prepare(test_dsf, augment=False)
test_ds_aug = prepare(test_dsf, augment=True)

In [ ]:
#Model with augmentation tested on augmented data set

model = tf.keras.Sequential([
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes_dsf)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


epochs=5
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/5
92/92 [==============================] - 90s 965ms/step - loss: 1.2346 - accuracy: 0.4925 - val_loss: 1.0329 - val_accuracy: 0.6158
Epoch 2/5
92/92 [==============================] - 89s 960ms/step - loss: 0.9338 - accuracy: 0.6386 - val_loss: 1.0323 - val_accuracy: 0.6158
Epoch 3/5
92/92 [==============================] - 89s 963ms/step - loss: 0.7509 - accuracy: 0.7190 - val_loss: 0.9586 - val_accuracy: 0.6485
Epoch 4/5
92/92 [==============================] - 89s 964ms/step - loss: 0.5546 - accuracy: 0.7977 - val_loss: 0.9201 - val_accuracy: 0.6458
Epoch 5/5
92/92 [==============================] - 89s 966ms/step - loss: 0.3186 - accuracy: 0.8890 - val_loss: 1.0879 - val_accuracy: 0.6349


In [ ]:
loss, acc = model.evaluate(test_ds)
loss, acc = model.evaluate(test_ds_aug)

12/12 [==============================] - 4s 316ms/step - loss: 1.0088 - accuracy: 0.6512


In [ ]:
train_ds = prepare(train_dsf, shuffle=True, augment=True)
val_ds = prepare(val_dsf, augment=True)
test_ds = prepare(test_dsf, augment=False)
test_ds_aug = prepare(test_dsf, augment=True)

In [ ]:
#Model with augmentation tested on augmented data set

model = tf.keras.Sequential([
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes_dsf)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


epochs=5
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/5
92/92 [==============================] - 96s 1s/step - loss: 1.2803 - accuracy: 0.4608 - val_loss: 1.1173 - val_accuracy: 0.5368
Epoch 2/5
92/92 [==============================] - 95s 1s/step - loss: 1.0035 - accuracy: 0.6005 - val_loss: 0.9997 - val_accuracy: 0.6458
Epoch 3/5
92/92 [==============================] - 95s 1s/step - loss: 0.8379 - accuracy: 0.6849 - val_loss: 0.8636 - val_accuracy: 0.6485
Epoch 4/5
92/92 [==============================] - 95s 1s/step - loss: 0.6735 - accuracy: 0.7446 - val_loss: 0.9905 - val_accuracy: 0.6322
Epoch 5/5
92/92 [==============================] - 96s 1s/step - loss: 0.5050 - accuracy: 0.8116 - val_loss: 0.9289 - val_accuracy: 0.6512


In [ ]:
#Results for RandomZoom
loss, acc = model.evaluate(test_ds)
loss, acc = model.evaluate(test_ds_aug)

12/12 [==============================] - 4s 312ms/step - loss: 0.8505 - accuracy: 0.6703


Add in Augmentation such as Saturtaion, Brightness, Shifting, 

In [ ]:
def tfds_imgen(ds, imgen, batch_size, batches_per):
    for images, labels in ds:
        flow_ = imgen.flow(images, labels, batch_size=batch_size)
        for _ in range(batches_per):
            yield next(flow_)

#Shifting Image
imgen = ImageDataGenerator(width_shift_range=0.3)

train_ds = prepare(train_dsf,shuffle=True)
train_dsa = tfds_imgen(
    prepare(train_dsf,shuffle=True).as_numpy_iterator(), imgen,
    batch_size=32, batches_per=32 // 32)

val_ds = prepare(train_dsf)
val_dsa = tfds_imgen(prepare(val_dsf).as_numpy_iterator(), imgen, batch_size=32, batches_per=32 // 32)

test_ds = prepare(test_dsf)
test_ds_aug = tfds_imgen(prepare(test_dsf).as_numpy_iterator(), imgen, batch_size=32, batches_per=32 // 32)

In [ ]:
model = tf.keras.Sequential([
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes_dsf)
])

#Generation of ImageGenerator
#model.fit_generator(alter_img, steps_per_epoch=5, ...)

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


epochs=5
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/5
92/92 [==============================] - 134s 1s/step - loss: 1.2942 - accuracy: 0.4438 - val_loss: 0.9973 - val_accuracy: 0.6090
Epoch 2/5
92/92 [==============================] - 121s 1s/step - loss: 0.9961 - accuracy: 0.6127 - val_loss: 0.8701 - val_accuracy: 0.7003
Epoch 3/5
92/92 [==============================] - 121s 1s/step - loss: 0.7972 - accuracy: 0.7044 - val_loss: 0.6537 - val_accuracy: 0.7684
Epoch 4/5
92/92 [==============================] - 125s 1s/step - loss: 0.5958 - accuracy: 0.7844 - val_loss: 0.4455 - val_accuracy: 0.8576
Epoch 5/5
92/92 [==============================] - 133s 1s/step - loss: 0.3663 - accuracy: 0.8740 - val_loss: 0.3416 - val_accuracy: 0.8750


In [ ]:
print("W/O Shift")
loss, acc = model.evaluate(test_ds)
loss, acc = model.evaluate(test_ds_aug)

W/O Shift
24/24 [==============================] - 12s 509ms/step - loss: 1.1503 - accuracy: 0.5967


In [ ]:
model = tf.keras.Sequential([
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes_dsf)
])

#Generation of ImageGenerator
#model.fit_generator(alter_img, steps_per_epoch=5, ...)

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


epochs=5
history = model.fit(
  train_dsa,
  validation_data=val_dsa,
  epochs=epochs
)

Epoch 1/5
92/92 [==============================] - 151s 2s/step - loss: 1.2669 - accuracy: 0.4639 - val_loss: 1.0519 - val_accuracy: 0.6076
Epoch 2/5


92/92 [==============================] - 0s 879us/step - loss: 1.2669 - accuracy: 0.4639


In [ ]:
print("With Shift")
loss, acc = model.evaluate(test_ds)
loss, acc = model.evaluate(test_ds_aug)

With Shift
12/12 [==============================] - 6s 480ms/step - loss: 1.0318 - accuracy: 0.5640


In [ ]:

#Brightness Image
imgen = ImageDataGenerator(brightness_range=[0.8,1.2])

train_ds = prepare(train_dsf,shuffle=True)
train_dsa = tfds_imgen(
    prepare(train_dsf,shuffle=True).as_numpy_iterator(), imgen,
    batch_size=32, batches_per=32 // 32)

val_ds = prepare(train_dsf)
val_dsa = tfds_imgen(prepare(val_dsf).as_numpy_iterator(), imgen, batch_size=32, batches_per=32 // 32)

test_ds = prepare(test_dsf)
test_ds_aug = tfds_imgen(prepare(test_dsf).as_numpy_iterator(), imgen, batch_size=32, batches_per=32 // 32)

In [ ]:
model = tf.keras.Sequential([
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes_dsf)
])

#Generation of ImageGenerator
#model.fit_generator(alter_img, steps_per_epoch=5, ...)

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


epochs=5
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
  steps_per_epoch = len(train_ds) // 32,
  validation_steps = len(val_ds) //32

)

Epoch 1/5
2/2 [==============================] - 4s 2s/step - loss: 4.2569 - accuracy: 0.2031 - val_loss: 5.4401 - val_accuracy: 0.1875
Epoch 2/5
2/2 [==============================] - 3s 2s/step - loss: 4.0718 - accuracy: 0.2500 - val_loss: 2.1841 - val_accuracy: 0.2188
Epoch 3/5
2/2 [==============================] - 3s 2s/step - loss: 2.0294 - accuracy: 0.1094 - val_loss: 1.7406 - val_accuracy: 0.2344
Epoch 4/5
2/2 [==============================] - 3s 2s/step - loss: 1.7719 - accuracy: 0.1719 - val_loss: 1.6034 - val_accuracy: 0.1719
Epoch 5/5
2/2 [==============================] - 3s 2s/step - loss: 1.5993 - accuracy: 0.2500 - val_loss: 1.5850 - val_accuracy: 0.2656


In [ ]:
print("W/O Brightness")
loss, acc = model.evaluate(test_ds)
loss, acc = model.evaluate(test_ds_aug)

W/O Brightness
12/12 [==============================] - 4s 299ms/step - loss: 1.5939 - accuracy: 0.2507


StopIteration: ignored

In [ ]:
model = tf.keras.Sequential([
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes_dsf)
])

#Generation of ImageGenerator
#model.fit_generator(alter_img, steps_per_epoch=5, ...)

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


epochs=5
history = model.fit(
  train_dsa,
  validation_data=val_dsa,
  epochs=epochs,
  steps_per_epoch = len(train_dsa) // 32,
  validation_steps = len(val_dsa) //32
)

Epoch 1/5
368/368 [==============================] - 462s 1s/step - loss: 7.4070 - accuracy: 0.5601 - val_loss: 1.2988 - val_accuracy: 0.4816
Epoch 2/5


368/368 [==============================] - 0s 148us/step - loss: 7.4070 - accuracy: 0.5601


In [ ]:
print("With Brightness")
loss, acc = model.evaluate(test_ds)
loss, acc = model.evaluate(test_ds_aug)

With Brightness
12/12 [==============================] - 4s 289ms/step - loss: 1.5044 - accuracy: 0.3161


StopIteration: ignored

In [ ]:
import cv2
from PIL import Image

def myFunc(image):
    image = np.array(image)
    hsv_image = cv2.cvtColor(image,cv2.COLOR_RGB2HSV)
    return Image.fromarray(hsv_image)

  #Brightness Image
imgen = ImageDataGenerator(preprocessing_function=myFunc)

train_ds = prepare(train_dsf,shuffle=True)
train_dsa = tfds_imgen(
    prepare(train_dsf,shuffle=True).as_numpy_iterator(), imgen,
    batch_size=32, batches_per=32 // 32)

val_ds = prepare(train_dsf)
val_dsa = tfds_imgen(prepare(val_dsf).as_numpy_iterator(), imgen, batch_size=32, batches_per=32 // 32)

test_ds = prepare(test_dsf)
test_ds_aug = tfds_imgen(prepare(test_dsf).as_numpy_iterator(), imgen, batch_size=32, batches_per=32 // 32)

In [ ]:
model = tf.keras.Sequential([
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes_dsf)
])

#Generation of ImageGenerator
#model.fit_generator(alter_img, steps_per_epoch=5, ...)

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


epochs=5
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs

)

Epoch 1/5
92/92 [==============================] - 125s 1s/step - loss: 1.3704 - accuracy: 0.4237 - val_loss: 1.0610 - val_accuracy: 0.6087
Epoch 2/5
92/92 [==============================] - 124s 1s/step - loss: 0.9892 - accuracy: 0.6151 - val_loss: 0.8572 - val_accuracy: 0.6792
Epoch 3/5
92/92 [==============================] - 125s 1s/step - loss: 0.7822 - accuracy: 0.6975 - val_loss: 0.6173 - val_accuracy: 0.7946
Epoch 4/5
92/92 [==============================] - 124s 1s/step - loss: 0.5615 - accuracy: 0.8004 - val_loss: 0.3669 - val_accuracy: 0.9091
Epoch 5/5
92/92 [==============================] - 124s 1s/step - loss: 0.3718 - accuracy: 0.8733 - val_loss: 0.2929 - val_accuracy: 0.9111


In [ ]:
print("W/O Color")
loss, acc = model.evaluate(test_ds)
loss, acc = model.evaluate(test_ds_aug)

In [ ]:
model = tf.keras.Sequential([
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes_dsf)
])

#Generation of ImageGenerator
#model.fit_generator(alter_img, steps_per_epoch=5, ...)

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


epochs=5
history = model.fit(
  train_dsa,
  validation_data=val_dsa,
  epochs=epochs
)

In [ ]:
print("With Color")
loss, acc = model.evaluate(test_ds)
loss, acc = model.evaluate(test_ds_aug)

Proof of Concept for Robustness


In [5]:
#This function may very from the data import source another version is shown above for using the ImageGenerator frunction from Keras 
#however I did face issues with this so the standard tensorflow augmentations were used
class standardize_and_robust():

#Step 1: scaling images
  def resize(image):
    
    IMG_SIZE = 180
    img_adj = []
    
    resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(IMG_SIZE, IMG_SIZE),
  layers.experimental.preprocessing.Rescaling(1./255)
])
    img_adj = resize_and_rescale(image)

    return img_adj

#Step 2: Expand dataset 
  def data_augmentation(image,label,augment_list): 
    img = []
    lab = []

    for aug in augment_list:
      for i in range(3):
        seed = (i, 0)  # tuple of size (2,)

        if aug == 'flip':
          data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2),
                                  ])
          img.append(data_augmentation)
          continue
        elif aug =='brightness':
          stateless_random_brightness = tf.image.stateless_random_brightness(
              image, max_delta=0.5, seed=seed)
          img.append(stateless_random_brightness)
          lab.append(label)
          continue
        elif aug == 'hue':
          stateless_random_hue = tf.image.stateless_random_hue(image,.5,seed=seed)
          img.append(stateless_random_hue)
          lab.append(label)
          continue
        elif aug == 'contrast':
          stateless_random_contrast = tf.image.stateless_random_contrast(image, .2,.5, seed=seed)
          img.append(stateless_random_contrast)
          lab.append(label)
          continue
        elif aug == 'crop':
          stateless_random_crop = tf.image.stateless_random_crop(image,shape=(1,2,3),seed=seed)
          img.append(stateless_random_crop)
          lab.append(label)
          continue
        elif aug == 'quality':
          stateless_random_jpeg_quality = tf.image.stateless_random_jpeg_quality(image,60, 95,seed=seed)
          img.append(stateless_random_jpeg_quality) 
          lab.append(label) 
          continue
        elif aug == 'staturation':
          stateless_random_saturation = tf.image.stateless_random_saturation(image,.5,.95,seed=seed)
          img.append(stateless_random_saturation) 
          lab.append(label)
          continue


    return img, lab

In [5]:
train_ds = prepare(train_dsf, shuffle=True)
val_ds = prepare(val_dsf,)
test_ds = prepare(test_dsf)


train_y = np.concatenate([y for x, y in train_ds], axis=0)
train_X = np.concatenate([x for x, y in train_ds], axis=0)
val_y  = np.concatenate([y for x, y in val_ds], axis=0)
val_X = np.concatenate([x for x, y in val_ds], axis=0)
test_y  = np.concatenate([y for x, y in test_ds], axis=0)
test_X = np.concatenate([x for x, y in test_ds], axis=0)


train_dsa = tf.data.Dataset.from_tensor_slices(standardize_and_robust.data_augmentation(train_X,train_y,['brightness']))
val_dsa = tf.data.Dataset.from_tensor_slices(standardize_and_robust.data_augmentation(val_X,val_y,['brightness']))
test_ds_aug = tf.data.Dataset.from_tensor_slices(standardize_and_robust.data_augmentation(test_X,test_y,['brightness']))

In [ ]:
model = tf.keras.Sequential([
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes_dsf)
])

#Generation of ImageGenerator
#model.fit_generator(alter_img, steps_per_epoch=5, ...)

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


epochs=5
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)


Epoch 1/5
92/92 [==============================] - 101s 1s/step - loss: 1.2667 - accuracy: 0.4625 - val_loss: 1.0583 - val_accuracy: 0.6322
Epoch 2/5
92/92 [==============================] - 100s 1s/step - loss: 0.9635 - accuracy: 0.6315 - val_loss: 0.9449 - val_accuracy: 0.6757
Epoch 3/5
92/92 [==============================] - 99s 1s/step - loss: 0.7406 - accuracy: 0.7245 - val_loss: 0.9078 - val_accuracy: 0.6921
Epoch 4/5
92/92 [==============================] - 104s 1s/step - loss: 0.5116 - accuracy: 0.8147 - val_loss: 1.0997 - val_accuracy: 0.5858
Epoch 5/5
92/92 [==============================] - 102s 1s/step - loss: 0.2956 - accuracy: 0.8978 - val_loss: 1.2043 - val_accuracy: 0.6512


In [ ]:
print("With Color")
loss, acc = model.evaluate(test_ds)
loss, acc = model.evaluate(test_ds_aug)

With Color
3/3 [==============================] - 12s 4s/step - loss: 1.4178 - accuracy: 0.5795


In [ ]:
model = tf.keras.Sequential([
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes_dsf)
])

#Generation of ImageGenerator
#model.fit_generator(alter_img, steps_per_epoch=5, ...)

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


epochs=5
history = model.fit(
  train_dsa,
  validation_data=val_dsa,
  epochs=epochs
)


Epoch 1/5


In [ ]:
print("With quality")
loss, acc = model.evaluate(test_ds)
loss, acc = model.evaluate(test_ds_aug)

In [6]:
train_ds = prepare(train_dsf, shuffle=True)
val_ds = prepare(val_dsf,)
test_ds = prepare(test_dsf)


train_y = np.concatenate([y for x, y in train_ds], axis=0)
train_X = np.concatenate([x for x, y in train_ds], axis=0)
val_y  = np.concatenate([y for x, y in val_ds], axis=0)
val_X = np.concatenate([x for x, y in val_ds], axis=0)
test_y  = np.concatenate([y for x, y in test_ds], axis=0)
test_X = np.concatenate([x for x, y in test_ds], axis=0)


train_dsa = tf.data.Dataset.from_tensor_slices(standardize_and_robust.data_augmentation(train_X,train_y,['hue']))
val_dsa = tf.data.Dataset.from_tensor_slices(standardize_and_robust.data_augmentation(val_X,val_y,['hue']))
test_ds_aug = tf.data.Dataset.from_tensor_slices(standardize_and_robust.data_augmentation(test_X,test_y,['hue']))

In [7]:
model = tf.keras.Sequential([
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes_dsf)
])

#Generation of ImageGenerator
#model.fit_generator(alter_img, steps_per_epoch=5, ...)

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


epochs=5
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)


Epoch 1/5
92/92 [==============================] - 101s 1s/step - loss: 1.3913 - accuracy: 0.4166 - val_loss: 1.1340 - val_accuracy: 0.5695
Epoch 2/5
92/92 [==============================] - 99s 1s/step - loss: 1.0361 - accuracy: 0.5869 - val_loss: 1.0707 - val_accuracy: 0.5777
Epoch 3/5
92/92 [==============================] - 99s 1s/step - loss: 0.8576 - accuracy: 0.6747 - val_loss: 0.9402 - val_accuracy: 0.6294
Epoch 4/5
92/92 [==============================] - 98s 1s/step - loss: 0.6705 - accuracy: 0.7425 - val_loss: 0.9329 - val_accuracy: 0.6512
Epoch 5/5
92/92 [==============================] - 98s 1s/step - loss: 0.4400 - accuracy: 0.8420 - val_loss: 1.1066 - val_accuracy: 0.6294


In [8]:
print("With/o hue")
loss, acc = model.evaluate(test_ds)
loss, acc = model.evaluate(test_ds_aug)

With/o hue
3/3 [==============================] - 10s 3s/step - loss: 3.4318 - accuracy: 0.3415


In [ ]:
model = tf.keras.Sequential([
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes_dsf)
])

#Generation of ImageGenerator
#model.fit_generator(alter_img, steps_per_epoch=5, ...)

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


epochs=5
history = model.fit(
  train_dsa,
  validation_data=val_dsa,
  epochs=epochs
)

Epoch 1/5


In [ ]:
print("With hue")
loss, acc = model.evaluate(test_ds)
loss, acc = model.evaluate(test_ds_aug)

In [6]:
train_ds = prepare(train_dsf, shuffle=True)
val_ds = prepare(val_dsf,)
test_ds = prepare(test_dsf)


train_y = np.concatenate([y for x, y in train_ds], axis=0)
train_X = np.concatenate([x for x, y in train_ds], axis=0)
val_y  = np.concatenate([y for x, y in val_ds], axis=0)
val_X = np.concatenate([x for x, y in val_ds], axis=0)
test_y  = np.concatenate([y for x, y in test_ds], axis=0)
test_X = np.concatenate([x for x, y in test_ds], axis=0)


train_dsa = tf.data.Dataset.from_tensor_slices(standardize_and_robust.data_augmentation(train_X,train_y,['staturation']))
val_dsa = tf.data.Dataset.from_tensor_slices(standardize_and_robust.data_augmentation(val_X,val_y,['staturation']))
test_ds_aug = tf.data.Dataset.from_tensor_slices(standardize_and_robust.data_augmentation(test_X,test_y,['staturation']))

In [7]:
model = tf.keras.Sequential([
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes_dsf)
])

#Generation of ImageGenerator
#model.fit_generator(alter_img, steps_per_epoch=5, ...)

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


epochs=5
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/5
92/92 [==============================] - 109s 1s/step - loss: 1.2504 - accuracy: 0.4745 - val_loss: 1.0864 - val_accuracy: 0.5668
Epoch 2/5
92/92 [==============================] - 107s 1s/step - loss: 0.9578 - accuracy: 0.6202 - val_loss: 0.9162 - val_accuracy: 0.6757
Epoch 3/5
92/92 [==============================] - 107s 1s/step - loss: 0.8115 - accuracy: 0.6870 - val_loss: 0.8949 - val_accuracy: 0.6676
Epoch 4/5
92/92 [==============================] - 104s 1s/step - loss: 0.6560 - accuracy: 0.7602 - val_loss: 0.9764 - val_accuracy: 0.6213
Epoch 5/5
92/92 [==============================] - 102s 1s/step - loss: 0.4426 - accuracy: 0.8426 - val_loss: 0.9803 - val_accuracy: 0.6649


In [8]:
print("With/o Staturation")
loss, acc = model.evaluate(test_ds)
loss, acc = model.evaluate(test_ds_aug)

With/o Staturation
3/3 [==============================] - 10s 3s/step - loss: 1.2222 - accuracy: 0.5804


In [ ]:
model = tf.keras.Sequential([
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes_dsf)
])

#Generation of ImageGenerator
#model.fit_generator(alter_img, steps_per_epoch=5, ...)

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


epochs=5
history = model.fit(
  train_dsa,
  validation_data=val_dsa,
  epochs=epochs
)

In [ ]:
print("With Staturation")
loss, acc = model.evaluate(test_ds)
loss, acc = model.evaluate(test_ds_aug)